In [1]:
from agm_te.dataset import DataSet
from agm_te.mlmodel import train_RNNDynamicsModel, init_dynamicsmodels_from_loaders
import numpy as np
from te_datasim.lineargaussian import BVLinearGaussianSimulator

In [2]:
import torch; torch.set_printoptions(sci_mode=None)
# Check if CUDA is available
if torch.cuda.is_available():
    compute_device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    compute_device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


### Initialize the Simulator

In [3]:
bivar = BVLinearGaussianSimulator()
print("analytical TE X -> Y: ", bivar.analytic_transfer_entropy('X', 'Y'))
print("analytical TE Y -> X: ", bivar.analytic_transfer_entropy('Y', 'X'))

bivar_data_dict = {'X':[], 'Y':[]}
for i in range(50):
    X, Y = bivar.simulate(1000, seed=i)
    bivar_data_dict['X'].append(X)
    bivar_data_dict['Y'].append(Y)

bivar_data = DataSet(bivar_data_dict)

analytical TE X -> Y:  0.0
analytical TE Y -> X:  0.1276


### Estimate $T_{X \to Y}$

In [4]:
dataloader_1, dataloader_2 = bivar_data.get_TE_dataloaders(device=compute_device, var_from='X', var_to='Y', batch_size=25)
model_1, model_2 = init_dynamicsmodels_from_loaders(dataloader_1, dataloader_2, hidden_size=2)
model_1, model_2, loss = train_RNNDynamicsModel(model_1, dataloader_1, model_2, dataloader_2, epochs=5000, learning_rate=1e-3)

Epoch [4900/5000], Model 1 Loss: 0.622958, Model 2 Loss: 0.625364, Estimated TE: -0.0024


### Estimate $T_{Y \to X}$

In [5]:
dataloader_1, dataloader_2 = bivar_data.get_TE_dataloaders(device=compute_device, var_from='Y', var_to='X', batch_size=25)
model_1, model_2 = init_dynamicsmodels_from_loaders(dataloader_1, dataloader_2, hidden_size=2)
model_1, model_2, loss = train_RNNDynamicsModel(model_1, dataloader_1, model_2, dataloader_2, epochs=5000, learning_rate=1e-3)

Epoch [4900/5000], Model 1 Loss: 0.784858, Model 2 Loss: 0.676142, Estimated TE: 0.1087
